In [60]:
### IMPORT PACKAGES ###

import pandas as pd
import numpy as np

# to make pretty plots
import plotly
import plotly.express as px
import plotly.graph_objects as go

# to play with time :-)
import datetime
import time

In [61]:
    ### IMPORT THE DATA ###
df = pd.read_csv('data/BPI_Challenge_2012-training.csv')
df_test = pd.read_csv('data/BPI_Challenge_2012-test.csv')

# Defining database-specific variables
case_column = "case concept:name"
event_column = "event concept:name"
timestamp_column = "event time:timestamp"
timeformat_timestamp = "%d-%m-%Y %H:%M:%S.%f"
lifecycle_column = 'event lifecycle:transition'
amount_column = 'case AMOUNT_REQ'
row_nr_column = 'row_nr'

In [62]:
# round the amount column on 10000
df[amount_column] = [round(x,-4) for x in df[amount_column]]
df_test[amount_column] = [round(x,-4) for x in df_test[amount_column]]

In [63]:
#make a list with all possible events
all_events=list(df[event_column].unique())

In [64]:
# get most common event
df_most_common_event =pd.DataFrame(df[event_column].value_counts())
df_most_common_event.reset_index(inplace=True)
most_common_event = df_most_common_event.iloc[0,0]

In [65]:
# additional columns that you want to include
additionalInfo = [amount_column]

# events that are influenced by concurency
eventList = ['A_SUBMITTED', 'A_PARTLYSUBMITTED', 'A_PREACCEPTED', 'O_SELECTED', 'A_FINALIZED', 'O_ACCEPTED']

# make a list of the columns you need
columns = [case_column, event_column, lifecycle_column]
columns.extend(additionalInfo)


# you can get rid of all the other columns, to make things faster
dataset = df[columns]
dataset_test = df_test[columns]
dataset[row_nr_column] = dataset.index
dataset_test[row_nr_column] = dataset_test.index

print(dataset.head())

   case concept:name      event concept:name event lifecycle:transition  \
0             173688             A_SUBMITTED                   COMPLETE   
1             173688       A_PARTLYSUBMITTED                   COMPLETE   
2             173688           A_PREACCEPTED                   COMPLETE   
3             173688  W_Completeren aanvraag                   SCHEDULE   
4             173691             A_SUBMITTED                   COMPLETE   

   case AMOUNT_REQ  row_nr  
0            20000       0  
1            20000       1  
2            20000       2  
3            20000       3  
4                0       4  


C:\Users\20203666\AppData\Local\Temp/ipykernel_13112/425723229.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[row_nr_column] = dataset.index
C:\Users\20203666\AppData\Local\Temp/ipykernel_13112/425723229.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_test[row_nr_column] = dataset_test.index


In [66]:
### STORAGE ###
# we want to keep track of things that happen within one case

# caseStorage stores: key=instance, values= [event, previousEvent, event_id, previousEvent_id, lifecycle, previousLifecycle, amount, predictedEvent]
# to look up the previous event in this instance, and it's predicted TimePassed
caseStorage = {}

# we also want to keep track of the predictionError
# errorStorage stores: key=unique integer, values= [previousEvent, previousLifecycle, predictionError,predictedEvent, event]
errorStorage = {}

# we also want to add a log feature => we don't have to save this seperatly, just needed to check if having it makes sense.
# how many events of the same activty are running at the same time might be interesting
# concurentEvents = {}

def getFeatures(case, event, row_nr):
    """
    This function extracts extra info of incomming event from the caseStorage
    """

    # if the case is already in storage, info of previous event is extracted from caseStorage
    if case in caseStorage:

        previousEvent = caseStorage[case][0]
        previousLifecycle = caseStorage[case][4]
        predictedEvent = caseStorage[case][7]
        previousEvent_id = caseStorage[case][2]
        previousRow_nr = caseStorage[case][8]

        # if prediction of previous event is correct, change predictionError to 1 and add the event to errorstorage
        if predictedEvent == event:
            predictionError = 1
        else:
            predictionError = 0
        errorStorage[previousRow_nr] = [previousEvent, previousLifecycle, predictionError,predictedEvent, event]

    # if it is a new case set info of previous event to None
    else:
        previousEvent = None
        previousEvent_id = None
        previousLifecycle = None

    if event in eventList:
        # you want to find how many times this activity occurs without any additional information
        howManyConcurentEvents = [x[0] for x in caseStorage.values()].count(event)
    else:
        howManyConcurentEvents = ''

    return previousEvent, previousEvent_id, previousLifecycle, howManyConcurentEvents



In [67]:
### THE MODEL ###
# We start with only one feature: the historical average time between two events based on the previous event
# You don't need a second model for the test set, since you train your model on the training-set and then use it on the test-set

predictionModel = {}

def getPrediction(event_id):
    """
    gets the prediction of a certain event_id
    """

    try:
        # get the event of an event_id with the highest occurrence
        return max(predictionModel[event_id],key=predictionModel[event_id].get)

    except:
        # if event_id not in predictionModel return the most frequent event in train data as prediction
        return most_common_event



def updatePredictionModel(this_event_id, event):
    """
    updates PredictionModel with the true event of a event_id
    """
    # makes dict with all events as key and 0 as value
    all_events_dict=dict([(event,0) for event in all_events])

    if this_event_id != None:

        # if event_id already in predictionModel update it by +1 to the true event
        try:
            predictionModel[this_event_id][event] +=1
        # if event_id not in predictionModel, add it to the dict with the all_events_dict as key and update it by +1 tot the true event
        except:
            predictionModel[this_event_id] = all_events_dict
            predictionModel[this_event_id][event] +=1



In [68]:
### RUNNING THE MODEL ###
# We have to run the model line by line, we can't do things in parallel, otherwise it's not on the fly

def processEvent(case, event, lifecycle, row_nr, amount=None):
    """
    the input is the the raw info of a case
    the function updates the predictionModel, caseStorage and the errorStorage
    """
    #gets info of the incomming event
    previousEvent, previousEvent_id, previousLifecycle, howManyConcurentEvents = getFeatures(case, event, row_nr)

    # makes the event_id based on the features and is used to make the prediction
    event_id = str(event)+'_XX_'+str(amount)+'_XX_'+str(previousEvent)+'_XX_'+str(previousLifecycle)+'_XX_'+str(howManyConcurentEvents)

    # gets the prediction of event_id from the predictionModel
    predictedEvent = getPrediction(event_id)

    # update storage
    caseStorage[case] = [event, previousEvent, event_id, previousEvent_id, lifecycle, previousLifecycle, amount, predictedEvent, row_nr]

    # update predictionModel
    updatePredictionModel(previousEvent_id, event)




### Run over all lines =>

# Start the stopwatch / counter
start_time = time.process_time()

# running the on-the-fly model for each row

[processEvent(case, event, amount, lifecycle, row_nr) for case, event, amount, lifecycle, row_nr in zip(dataset[case_column],dataset[event_column], dataset[lifecycle_column], dataset[row_nr_column], dataset[amount_column])]

# Stop the stopwatch / counter
end_time = time.process_time()

print("dict: Elapsed time during the whole program in seconds:", end_time-start_time)


dict: Elapsed time during the whole program in seconds: 14.625


In [76]:
df

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,0,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256
...,...,...,...,...,...,...,...
214372,38835094290529,201854,2012-01-18T02:09:07.029+01:00,50000,O_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214373,38835094290528,201854,2012-01-18T02:09:07.029+01:00,50000,A_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214374,38835094290530,201854,2012-01-18T02:09:07.029+01:00,50000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 15:30:23.187
214375,35858681954366,199678,2012-01-10T19:16:52.800+01:00,30000,W_Nabellen offertes,START,14-03-2012 15:36:15.299


In [77]:
len(errorStorage)

45205

In [70]:
# count keeps track on how many the model was correct and divides by the amount of predictions made
count=0
for i in errorStorage.values():
    count+=i[2]
len_set = len(errorStorage)
print(f'train accuracy:{count/len_set}')


train accuracy:0.8166329913490398


In [71]:
df

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,0,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256
...,...,...,...,...,...,...,...
214372,38835094290529,201854,2012-01-18T02:09:07.029+01:00,50000,O_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214373,38835094290528,201854,2012-01-18T02:09:07.029+01:00,50000,A_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214374,38835094290530,201854,2012-01-18T02:09:07.029+01:00,50000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 15:30:23.187
214375,35858681954366,199678,2012-01-10T19:16:52.800+01:00,30000,W_Nabellen offertes,START,14-03-2012 15:36:15.299


In [72]:
predictionModel

{'A_SUBMITTED_XX_20000_XX_None_XX_None_XX_0': {'A_SUBMITTED': 0,
  'A_PARTLYSUBMITTED': 2354,
  'A_PREACCEPTED': 0,
  'W_Completeren aanvraag': 0,
  'A_DECLINED': 0,
  'W_Afhandelen leads': 0,
  'A_ACCEPTED': 0,
  'O_SELECTED': 0,
  'A_FINALIZED': 0,
  'O_CREATED': 0,
  'O_SENT': 0,
  'W_Nabellen offertes': 0,
  'O_CANCELLED': 0,
  'A_CANCELLED': 0,
  'W_Beoordelen fraude': 0,
  'O_SENT_BACK': 0,
  'W_Valideren aanvraag': 0,
  'W_Nabellen incomplete dossiers': 0,
  'O_ACCEPTED': 0,
  'A_APPROVED': 0,
  'A_ACTIVATED': 0,
  'A_REGISTERED': 0,
  'O_DECLINED': 0,
  'W_Wijzigen contractgegevens': 0},
 'A_PARTLYSUBMITTED_XX_20000_XX_A_SUBMITTED_XX_COMPLETE_XX_0': {'A_SUBMITTED': 0,
  'A_PARTLYSUBMITTED': 0,
  'A_PREACCEPTED': 992,
  'W_Completeren aanvraag': 0,
  'A_DECLINED': 392,
  'W_Afhandelen leads': 816,
  'A_ACCEPTED': 0,
  'O_SELECTED': 0,
  'A_FINALIZED': 0,
  'O_CREATED': 0,
  'O_SENT': 0,
  'W_Nabellen offertes': 0,
  'O_CANCELLED': 0,
  'A_CANCELLED': 0,
  'W_Beoordelen fraude': 

In [73]:
# empty errorStorage and run model on test set
errorStorage = {}
[processEvent(case, event, amount, lifecycle, row_nr) for case, event, amount, lifecycle, row_nr in zip(dataset_test[case_column],dataset_test[event_column], dataset_test[lifecycle_column], dataset_test[row_nr_column], dataset_test[amount_column])]

# count keeps track on how many the model was correct and divides by the amount of predictions made

count=0
for i in errorStorage.values():
    count+=i[2]
len_set = len(errorStorage)
print(f'test accuracy:{count/len_set}')

test accuracy:0.8201083950890389


In [74]:
errorStorage

{0: ['A_SUBMITTED', 'COMPLETE', 1, 'A_PARTLYSUBMITTED', 'A_PARTLYSUBMITTED'],
 1: ['A_PARTLYSUBMITTED', 'COMPLETE', 0, 'W_Afhandelen leads', 'A_DECLINED'],
 3: ['A_SUBMITTED', 'COMPLETE', 1, 'A_PARTLYSUBMITTED', 'A_PARTLYSUBMITTED'],
 4: ['A_PARTLYSUBMITTED', 'COMPLETE', 1, 'A_PREACCEPTED', 'A_PREACCEPTED'],
 5: ['A_PREACCEPTED',
  'COMPLETE',
  1,
  'W_Completeren aanvraag',
  'W_Completeren aanvraag'],
 7: ['A_SUBMITTED', 'COMPLETE', 1, 'A_PARTLYSUBMITTED', 'A_PARTLYSUBMITTED'],
 8: ['A_PARTLYSUBMITTED', 'COMPLETE', 0, 'W_Afhandelen leads', 'A_DECLINED'],
 10: ['A_SUBMITTED', 'COMPLETE', 1, 'A_PARTLYSUBMITTED', 'A_PARTLYSUBMITTED'],
 11: ['A_PARTLYSUBMITTED', 'COMPLETE', 1, 'A_PREACCEPTED', 'A_PREACCEPTED'],
 12: ['A_PREACCEPTED',
  'COMPLETE',
  1,
  'W_Completeren aanvraag',
  'W_Completeren aanvraag'],
 14: ['A_SUBMITTED', 'COMPLETE', 1, 'A_PARTLYSUBMITTED', 'A_PARTLYSUBMITTED'],
 15: ['A_PARTLYSUBMITTED',
  'COMPLETE',
  0,
  'A_PREACCEPTED',
  'W_Afhandelen leads'],
 17: ['A_SUB

In [75]:
df_test

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,44964012621824,206324,2012-02-03T17:17:11.047+01:00,0,A_SUBMITTED,COMPLETE,03-02-2012 17:17:11.047
1,44964012621825,206324,2012-02-03T17:17:11.047+01:00,0,A_PARTLYSUBMITTED,COMPLETE,03-02-2012 17:17:11.323
2,44964012621826,206324,2012-02-03T17:17:11.047+01:00,0,A_DECLINED,COMPLETE,03-02-2012 17:17:42.964
3,44968307589120,206327,2012-02-03T17:23:41.949+01:00,10000,A_SUBMITTED,COMPLETE,03-02-2012 17:23:41.949
4,44968307589121,206327,2012-02-03T17:23:41.949+01:00,10000,A_PARTLYSUBMITTED,COMPLETE,03-02-2012 17:23:42.504
...,...,...,...,...,...,...,...
47818,54666343743523,213276,2012-02-27T14:12:41.868+01:00,20000,W_Nabellen incomplete dossiers,START,14-03-2012 15:59:28.309
47819,54666343743524,213276,2012-02-27T14:12:41.868+01:00,20000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 16:00:09.680
47820,49495203119136,209595,2012-02-15T10:10:36.503+01:00,10000,W_Nabellen offertes,START,14-03-2012 16:02:03.883
47821,52342766436386,211624,2012-02-21T23:38:40.044+01:00,40000,W_Nabellen incomplete dossiers,START,14-03-2012 16:04:46.192
